<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/TrajectoryOfSphereWithSpin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [26]:
def Cd(Re):
    a=Re/5
    b=Re/2.63e5
    c=Re/1e6
    return 24/Re + 2.6*a/(1+a**1.52) +0.411*b**(-10)/(1+b**(-10)) + 0.25*c/(1+c)

In [27]:
x = np.logspace(2,7,500)
Cd_vals = Cd(x)

In [28]:
fig = make_subplots()
fig.add_scatter(x=x,y=Cd_vals, mode='lines')
fig.update_xaxes(type='log')
fig.update_layout(width=800,height=600, template='plotly_dark')

In [36]:
rho = 1.29 #kg/m3
mu = 1.8e-5
d = 0.22
A = np.pi*(d**2)/4
g = 9.81
m= 0.42
s= np.array([0,0,10])
s_mag = np.linalg.norm(s)


def rhs(t, xv):
    v=xv[3:]
    v_mag = np.linalg.norm(v)
    Re = rho*v_mag*d/mu
    Re_hi = rho*(v_mag+s_mag*d/2)*d/mu
    Re_lo =  rho*(v_mag-s_mag*d/2)*d/mu
    dCd_dRe = Cd(Re_hi)-Cd(Re_lo)/(Re_hi - Re_lo)
    Cl=0.1
    dxdt = v
    dvdt = np.array([0, 0, -g]) - rho*Cd(Re)*A*v_mag/2*v/m + rho*Cl*A* d/2 * np.cross(s,v)/m
    return np.concatenate([dxdt,dvdt])

def event(t, xv):
    return xv[2]

event.terminal=True
event.direction=-1

In [37]:
v_mag_mph = 80
angle = 40
side_angle = 15
v_mag = v_mag_mph * 0.44704
theta = angle*np.pi/180
phi = side_angle*np.pi/180
v = v_mag*np.array([np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), np.sin(theta)])

In [38]:
tend=10
res = solve_ivp(rhs, (0,tend), np.concatenate([[0,0,0],v]), method='Radau', dense_output=True, events=event)

In [8]:
t_plot = np.linspace(0,res.t_events[0][0],50)
x,y=res.sol(t_plot)[:2]
fig2=make_subplots()
fig2.add_scatter(x=x,y=y,mode='markers')
fig2.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig2.update_layout(width=800,height=600, template='plotly_dark')